In [1]:
import custom_coco_helper as helper
import os
import json

In [2]:
# Constants

DATA_DIR = 'data'

# dir for raw images
DOWNLOAD_DIR = 'raw'
IMAGE_PATH = os.path.join(os.getcwd(), DATA_DIR, DOWNLOAD_DIR)

# dir for intermediate files
IN_PROGRESS_DIR = 'in_progress'
IN_PROGRESS_PATH = os.path.join(os.getcwd(), DATA_DIR, IN_PROGRESS_DIR)

# intermediate files
FINISHED_PATH = os.path.join(IN_PROGRESS_PATH, 'finished.txt')
ANNOTATIONS_PATH = os.path.join(IN_PROGRESS_PATH, 'annotations.txt')
IMG_INFO_PATH = os.path.join(IN_PROGRESS_PATH, 'img_info.txt')

In [3]:
predictor = helper.MyPredictor(IMAGE_PATH)

In [4]:
categories = [
    {
        'id': 1,
        'name': 'people',
        'supercategory': 'people',
    },
]

In [5]:
next_seg_id = 1
annotations = []
imgs_info = []
my_instances = []

# Reading in previous processed img paths
with open(FINISHED_PATH, 'r') as f:
    finished_img_paths = f.readlines()

finished_file = open(FINISHED_PATH, 'a')
annotations_file = open(ANNOTATIONS_PATH, 'a')
img_info_file = open(IMG_INFO_PATH, 'a')

# Process 1 image per iteration
for i in range(2):
    # Skip if img was processed before
    img_path = predictor.get_img_path(i)
    if img_path in finished_img_paths:
        continue
        
    img = predictor.read_img(i)
    results = predictor.inference(img)
    next_seg_id, instances = helper.MyInstance.create_instances(i, next_seg_id, img_path, results, keep_cats=[0])
    imgs_info.append(instances[0].coco_img_info())
    
    # Display each people instance one by one for relabelling
    for instance in instances:
        predictor.visualize(img, instance)
        my_instances.append(instance)
        annotation = instance.coco_annotation()
        if annotation is not None:
            annotations_file.write(json.dumps(annotation) + '\n')
            
    img_info_file.write(json.dumps(instances[0].coco_img_info()) + '\n')
    finished_file.write(img_path + '\n')
    
img_info_file.close()
finished_file.close()
annotations_file.close()